# Cargar datos

In [1]:
import pandas as pd               # Librería para manipulación de datos en DataFrames
from pathlib import Path          # Manejo de rutas de forma multiplataforma

# Ruta absoluta a la carpeta que contiene los archivos .parquet
carpeta = Path("c:/Workspace/mlops_fleca_project/data/raw")

# Buscar todos los archivos con extensión .parquet dentro de la carpeta
ficheros_parquet = list(carpeta.glob("*.parquet"))

# Verificar si se encontraron archivos
if not ficheros_parquet:
    print("⚠️ No se encontraron archivos .parquet en la carpeta especificada.")
else:
    # Imprimir los archivos encontrados para verificación
    print(f"Archivos encontrados: {len(ficheros_parquet)}")
    for f in ficheros_parquet:
        print(f" - {f.name}")

    # Leer y concatenar todos los archivos .parquet en un solo DataFrame
    try:
        df = pd.concat(
            [pd.read_parquet(fichero, engine="pyarrow") for fichero in ficheros_parquet],
            ignore_index=True  # Evita mantener los índices originales de cada archivo
        )

        # Mostrar las primeras filas del DataFrame resultante
        print("\n✅ Datos cargados correctamente:")
        print(df.head())
    except Exception as e:
        print(f"❌ Error al leer los archivos .parquet: {e}")

Archivos encontrados: 4
 - raw_data_bq_forecasting_20250714.parquet
 - row_data_forecasting_2023-2025_02-000000000000.parquet
 - row_data_forecasting_2023-2025_02-000000000001.parquet
 - row_data_forecasting_2023-2025_02-000000000002.parquet
❌ Error al leer los archivos .parquet: data type 'dbdate' not understood


In [9]:

#analizamos valorese estadísticos de las columnas
df.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295979 entries, 0 to 295978
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fecha           295979 non-null  object 
 1   n_factura       295979 non-null  object 
 2   producto        295979 non-null  object 
 3   familia         295978 non-null  object 
 4   cantidad        295979 non-null  float64
 5   base_imponible  293732 non-null  float64
 6   total           293732 non-null  float64
dtypes: float64(3), object(4)
memory usage: 15.8+ MB


In [10]:
# pasamos las fechas a datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d %H:%M:%S.%f')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295979 entries, 0 to 295978
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           295979 non-null  datetime64[ns]
 1   n_factura       295979 non-null  object        
 2   producto        295979 non-null  object        
 3   familia         295978 non-null  object        
 4   cantidad        295979 non-null  float64       
 5   base_imponible  293732 non-null  float64       
 6   total           293732 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 15.8+ MB


In [11]:
# nos quedamos con las columnas que nos interesan:
# fecha, familia, cantidad, base_imponible
df = df[['fecha', 'familia', 'cantidad', 'base_imponible','total']]
df.head()


,fecha,familia,cantidad,base_imponible,total
0,2023-11-09,PAN,1.0,NaN,NaN
1,2023-11-07,PAN,1.0,NaN,NaN
2,2023-11-08,PAN,1.0,NaN,NaN
3,2023-07-24,PAN,0.8,0.06,0.06
4,2023-08-13,PAN,1.0,0.08,0.08


In [12]:
# renombramos la columna cantidad a unidades_vendidas
df.rename(columns={'cantidad': 'unidades_vendidas'}, inplace=True)
# r

In [15]:
# Guardamos el dataframe con el nombre df_fleca en un nuevo archivo parquet en la carpeta interim que se encuentra en la carpeta data
# del proyecto

carpeta = Path("c:/Workspace/mlops_fleca_project/data")
df.to_parquet(carpeta / "interim" / "df_fleca", index=False)